In [9]:
from datasets import load_dataset, DatasetDict
import numpy as np
import os
# libraries for Transformer
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import wandb
import evaluate
from dotenv import load_dotenv

In [10]:
load_dotenv()

True

In [11]:
api_key=os.getenv("WANDB_API_KEY")
run_name=os.getenv("WANDB_RUN_NAME")
# for development mode, set to true, right now imcreating pipeline, so take a data portion
testing= os.getenv('TESTING','False').lower() # optional 
dataset_name=os.getenv("DATASET_NAME")

if api_key is None or dataset_name is None:
    raise ValueError ('please...')

In [12]:
wandb.login(key=api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\fakih\_netrc
wandb: Currently logged in as: fakihakhan96 (fakihakhan96-nust) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [42]:
run=wandb.init(project ='bert-tiny-training',name=run_name)

In [ ]:
datasets= load_dataset(dataset_name)
datasets = DatasetDict({
'train':datasets['train'],
'test':datasets['test']
})

README.md:   0%|          | 0.00/480 [00:00<?, ?B/s]

d:\ML OPS\training-pipeline\env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\fakih\.cache\huggingface\hub\datasets--moizasghar--imdb-small-5k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


train-00000-of-00001.parquet:   0%|          | 0.00/3.19M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


test-00000-of-00001.parquet:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3750 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [15]:
print(f"data size :{datasets['train'].num_rows}")

data size :3750


In [16]:
testing

'true'

In [17]:
if testing =='true':
    datasets['train']=datasets['train'].shuffle(seed=42).select(range(10))
    datasets['test']=datasets['test'].shuffle(seed=42).select(range(1))
    # reducing datasize for testing pipeline 

In [21]:
tokenizer=AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")

In [24]:
def tokenize_fn(examples):
    return tokenizer(examples['text'],padding="max_length",truncation=True,max_length=256)

In [34]:
encoded_datasets=datasets.map(tokenize_fn,batched=True)
encoded_datasets.set_format(type='torch',columns=['input_ids','attention_mask','label'])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [26]:
model= AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny",num_labels=2)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [27]:
accuracy=evaluate.load("accuracy")
f1=evaluate.load("f1")

In [28]:
def compute_metrics(sample):
    logits,labels=sample
    pred=np.argmax(logits,axis=-1)
    acc=accuracy.compute(predictions=pred,references=labels)
    f1_score=f1.compute(predictions=pred,references=labels,average='weighted')
    return{"accuracy":acc["accuracy"],"f1":f1_score("f1")}
    

In [33]:
training_arg=TrainingArguments(
    warmup_ratio=0.1,
    lr_scheduler_type='cosine', #help training
    learning_rate=2e-5,
    max_grad_norm=1,
    output_dir="./results",
    report_to='wandb',
    run_name=run_name,
    save_total_limit=3
    )


In [37]:
trainer =Trainer(model=model,args=training_arg,
                 train_dataset=encoded_datasets['train'],
                 eval_dataset=encoded_datasets['test'],
                 compute_metrics=compute_metrics,
                 callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
                 )

In [38]:
trainer.train()

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


AssertionError: EarlyStoppingCallback requires metric_for_best_model to be defined

In [40]:
model.config.id2label={0:'negative',1:'positive'}
model.config.label2id={'negative':0,'positive':1}
model.save_pretrained("bert-tiny-sentiment")
tokenizer.save_pretrained('bert-tiny-sentiment')

('bert-tiny-sentiment\\tokenizer_config.json',
 'bert-tiny-sentiment\\special_tokens_map.json',
 'bert-tiny-sentiment\\vocab.txt',
 'bert-tiny-sentiment\\added_tokens.json',
 'bert-tiny-sentiment\\tokenizer.json')